In [1]:
import zmq

# Create a ZeroMQ context
context = zmq.Context()

# Create a socket to communicate with the server
socket = context.socket(zmq.REQ)
server_ip = "10.203.232.86"
server_port = "5555"  # Make sure this matches the server's port
server_address = f"tcp://{server_ip}:{server_port}"

# Connect to the server
socket.connect(server_address)

# Send a string message to the server
message = "Hello, Server!"
print(f"Sending message to server: {message}")
socket.send_string(message)

# Wait for a reply from the server
reply = socket.recv_string()
print(f"Received reply from server: {reply}")

# Clean up
socket.close()
context.term()

Sending message to server: Hello, Server!


KeyboardInterrupt: 

In [ ]:
import zmq

context = zmq.Context()
socket = context.socket(zmq.SUB)
broadcast_ip = "192.168.0.15"
broadcast_port = "5588"  # Make sure this matches the broadcaster's port
broadcast_address = f"tcp://{broadcast_ip}:{broadcast_port}"

# Connect to the broadcast
socket.connect(broadcast_address)

# Subscribe to all messages
socket.setsockopt(zmq.SUBSCRIBE, b"")
socket.setsockopt(zmq.RCVTIMEO, 1000)

print("Listening for broadcast messages...")

try:
    while True:
        try:
            # Receive the message
            topic, message = socket.recv_multipart()
            # Deserialize the numpy array
            decode_topic = topic.decode("utf-8")
            decode_msg = message.decode("utf-8")

            print(f"Received array: {decode_topic}, {decode_msg}")
        except zmq.Again as e:
            # Handle the timeout exception
            print("No message received within the timeout period.")
except KeyboardInterrupt:
    print("Stopped listening.")

# Clean up
socket.close()
context.term()

Exception ignored in: <function Context.__del__ at 0x7f9633c73f70>
Traceback (most recent call last):
  File "/home/jeremy/anaconda3/envs/posehub/lib/python3.9/site-packages/zmq/sugar/context.py", line 95, in __del__
    self.destroy()
  File "/home/jeremy/anaconda3/envs/posehub/lib/python3.9/site-packages/zmq/sugar/context.py", line 275, in destroy
    self.term()
  File "/home/jeremy/anaconda3/envs/posehub/lib/python3.9/site-packages/zmq/sugar/context.py", line 217, in term
    super().term()
  File "zmq/backend/cython/context.pyx", line 83, in zmq.backend.cython.context.Context.term
  File "zmq/backend/cython/checkrc.pxd", line 13, in zmq.backend.cython.checkrc._check_rc
KeyboardInterrupt: 


Listening for broadcast messages...
No message received within the timeout period.
No message received within the timeout period.
No message received within the timeout period.
No message received within the timeout period.
No message received within the timeout period.
No message received within the timeout period.
No message received within the timeout period.
No message received within the timeout period.
No message received within the timeout period.
No message received within the timeout period.
No message received within the timeout period.


In [10]:
import scipy

print(scipy.__version__)

from scipy.spatial.transform import Rotation as R
import numpy as np

r = R.from_quat([0, 0, np.sin(np.pi/4), np.cos(np.pi/4)])

1.13.1


In [1]:
import sys
from PyQt5.QtCore import QUrl, QSize, Qt
from PyQt5.QtGui import QColor, QVector3D
from PyQt5.QtWidgets import QApplication
from PyQt5.Qt3DExtras import (
    Qt3DWindow,
    QOrbitCameraController,
    QPhongMaterial,
    QMesh,
    QTransform
)
from PyQt5.Qt3DCore import QEntity

# Create the main application instance
app = QApplication(sys.argv)

# Initialize the 3D window
view = Qt3DWindow()
view.defaultFrameGraph().setClearColor(QColor(30, 30, 30))  # set a dark gray background
container = view.createWindowContainer()
container.setMinimumSize(QSize(800, 600))
container.setMaximumSize(QSize(800, 600))
container.setFocusPolicy(Qt.StrongFocus)

# Create the root entity which acts as the parent for all 3D objects in the scene
rootEntity = QEntity()

# Set up the camera
camera = view.camera()
camera.lens().setPerspectiveProjection(45.0, 16/9, 0.1, 1000.0)
camera.setPosition(QVector3D(0, 0, 20))
camera.setViewCenter(QVector3D(0, 0, 0))

# Add an orbit camera controller so that the user can interact with the scene
camController = QOrbitCameraController(rootEntity)
camController.setLinearSpeed(50.0)
camController.setLookSpeed(180.0)
camController.setCamera(camera)

# Define the file path to your OBJ file. Replace the path with the actual location of your model.
obj_file_url = QUrl.fromLocalFile("path/to/your/model.obj")

# ------------------------------------------------------------------------
# First entity: Load the OBJ file and place it at a first location (e.g., -5 units on the X-axis)
entity1 = QEntity(rootEntity)
mesh1 = QMesh()
mesh1.setSource(obj_file_url)
transform1 = QTransform()
transform1.setTranslation(QVector3D(-5, 0, 0))  # position the first model at (-5, 0, 0)
material1 = QPhongMaterial(rootEntity)
material1.setDiffuse(QColor(200, 100, 100))  # optional: set a diffuse color

# Add the components to the first entity
entity1.addComponent(mesh1)
entity1.addComponent(transform1)
entity1.addComponent(material1)

# ------------------------------------------------------------------------
# Second entity: Load the same OBJ file again and place it at a different location (e.g., +5 units on the X-axis)
entity2 = QEntity(rootEntity)
mesh2 = QMesh()
mesh2.setSource(obj_file_url)
transform2 = QTransform()
transform2.setTranslation(QVector3D(5, 0, 0))  # position the second model at (5, 0, 0)
material2 = QPhongMaterial(rootEntity)
material2.setDiffuse(QColor(100, 100, 200))  # optional: set a different diffuse color for variety

# Add the components to the second entity
entity2.addComponent(mesh2)
entity2.addComponent(transform2)
entity2.addComponent(material2)

# ------------------------------------------------------------------------
# Set the root entity of the scene and display the window
view.setRootEntity(rootEntity)
view.show()

# Execute the application
sys.exit(app.exec_())


ImportError: /lib/x86_64-linux-gnu/libgssapi_krb5.so.2: undefined symbol: krb5_ser_context_init, version krb5_3_MIT